In [4]:
import faiss
import numpy as np

# 1. 数据准备（模拟数据，维度为128，数据库向量数10000，查询向量数10）
dim = 128  # 向量维度
db_size = 10000  # 数据库向量数量
query_size = 10  # 查询向量数量
np.random.seed(42)  # 固定随机种子，确保结果可复现

db_vectors = np.random.random((db_size, dim)).astype('float32')  # 数据库向量（必须为float32类型）
query_vectors = np.random.random((query_size, dim)).astype('float32')  # 查询向量

# 2. 索引初始化（三种方式，按需选择）

# 方式1：IndexFlatL2（L2 距离，最常用）
index_l2 = faiss.IndexFlatL2(dim)

# 方式2：IndexFlatIP（内积，用于点积相似度或余弦相似度）
index_ip = faiss.IndexFlatIP(dim)

# 方式3：IndexFlat + 归一化实现余弦相似度（推荐）
# 某些 FAISS 版本没有 IndexFlatCOSINE，因此余弦相似度需手动归一化并使用 IndexFlatIP
db_vectors_norm = db_vectors.copy()
faiss.normalize_L2(db_vectors_norm)  # 手动归一化为单位向量
index_cosine = faiss.IndexFlatIP(dim)
index_cosine.add(db_vectors_norm)

# 3. 添加向量到索引（IndexFlat 系列都是直接 add）
index_l2.add(db_vectors)
print(f"索引中已添加的向量数量：{index_l2.ntotal}")

# 4. 执行检索（核心参数：query 向量、Top-K）
k = 5  # 返回最近邻数量
distances, indices = index_l2.search(query_vectors, k)

# 5. 结果解读
print("查询向量 0 的检索结果：")
print(f"距离：{distances[0]}")  # 与数据库向量的 L2 距离
print(f"索引：{indices[0]}")    # 对应数据库向量的编号

索引中已添加的向量数量：10000
查询向量 0 的检索结果：
距离：[13.346962 14.548449 14.708296 14.756075 14.837142]
索引：[8769 9385   82 5125 9571]


In [5]:
import faiss
import numpy as np
import time

print("FAISS version:", faiss.__version__)

# 1. 数据准备
dim = 128
db_size = 10000
query_size = 50

np.random.seed(42)
db_vectors = np.random.random((db_size, dim)).astype('float32')
query_vectors = np.random.random((query_size, dim)).astype('float32')

# 2. 创建索引：L2、IP
index_l2 = faiss.IndexFlatL2(dim)
index_ip = faiss.IndexFlatIP(dim)

# 3. 余弦相似度：手动归一化 + IP
db_vectors_norm = db_vectors.copy()
query_vectors_norm = query_vectors.copy()
faiss.normalize_L2(db_vectors_norm)
faiss.normalize_L2(query_vectors_norm)

index_cos = faiss.IndexFlatIP(dim)

# 添加向量
index_l2.add(db_vectors)
index_ip.add(db_vectors)
index_cos.add(db_vectors_norm)

print("三个索引均已添加向量：", index_l2.ntotal)

# 4. 检索
k = 10

# ---- L2 ----
t0 = time.time()
dist_l2, idx_l2 = index_l2.search(query_vectors, k)
t1 = time.time()
print(f"IndexFlatL2 检索时间：{t1 - t0:.4f} 秒")

# ---- IP ----
t0 = time.time()
dist_ip, idx_ip = index_ip.search(query_vectors, k)
t1 = time.time()
print(f"IndexFlatIP 检索时间：{t1 - t0:.4f} 秒")

# ---- COSINE ----
t0 = time.time()
dist_cos, idx_cos = index_cos.search(query_vectors_norm, k)
t1 = time.time()
print(f"Cosine（归一化+IP）检索时间：{t1 - t0:.4f} 秒")

# 5. Top-K 重合率对比（第 0 个查询向量）
print("\n=== 第 0 个查询向量的 Top-K 对比 ===")
print("L2 Top-10：", idx_l2[0])
print("IP  Top-10：", idx_ip[0])
print("COS Top-10：", idx_cos[0])

def overlap(a, b):
    return len(set(a) & set(b))

print(f"\nL2 vs IP   重合度：{overlap(idx_l2[0], idx_ip[0])}/10")
print(f"L2 vs COS  重合度：{overlap(idx_l2[0], idx_cos[0])}/10")
print(f"IP vs COS  重合度：{overlap(idx_ip[0], idx_cos[0])}/10")

FAISS version: 1.13.2
三个索引均已添加向量： 10000
IndexFlatL2 检索时间：0.0019 秒
IndexFlatIP 检索时间：0.0019 秒
Cosine（归一化+IP）检索时间：0.0015 秒

=== 第 0 个查询向量的 Top-K 对比 ===
L2 Top-10： [8769 9385   82 5125 9571 3491 6267 3948 4436 1056]
IP  Top-10： [6267 8290 1274 5831 2135 1323 9231 2825 2099 1717]
COS Top-10： [6267 9385 9571   82 8769 9352 4195 3491 3848 2825]

L2 vs IP   重合度：1/10
L2 vs COS  重合度：6/10
IP vs COS  重合度：2/10
